# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to AIC and BIC!

## Summary

You will be able to:
- Build a linear regression model with interactions and polynomial features 
- Use AIC and BIC to select the best value for the regularization parameter 


## Let's get started!

Import all the necessary packages.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import scale
from sklearn.preprocessing import PolynomialFeatures

Load the data.

In [2]:
df = pd.read_csv("ames.csv")

In [3]:
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

## Look at a baseline housing data model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Split the data into target (`y`) and predictors (`X`) -- ensure these both are DataFrames 
- Scale all the predictors using `scale`. Convert these scaled features into a DataFrame 
- Build at a baseline model using *scaled variables* as predictors. Use 5-fold cross-validation (set `random_state` to 1) and use the $R^2$ score to evaluate the model 

In [4]:
# Your code here
y = df["SalePrice"]
X = df[[column for column in df.columns if column != "SalePrice"]]
X_scaled = scale(X)
columns = [column for column in df.columns if column != "SalePrice"]

In [5]:
X = pd.DataFrame(X_scaled, columns = X.columns)

In [6]:
kfold = KFold(5 ,random_state = 1, shuffle = True)
linreg = LinearRegression()

In [7]:
kfold

KFold(n_splits=5, random_state=1, shuffle=True)

In [8]:
basemodel = cross_val_score(linreg, X, y, scoring = "r2", cv = kfold)
basemodel

array([0.811491  , 0.62555727, 0.79006425, 0.70807127, 0.82719172])

## Include interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Next, evaluate that model using 5-fold cross-validation and store the $R^2$ to compare it with the baseline model.

Print the 7 most important interactions.

In [9]:
# Your code here
combinations_list = list(combinations(columns, 2))
interactions_list = []
data = X.copy()
for combination in combinations_list:
    data["interaction"] = data[combination[0]]*data[combination[1]]
    score = np.mean(cross_val_score(linreg, data, y, scoring = "r2", cv = kfold))
    interactions_list.append([score, combination[0], combination[1]])

results = sorted(interactions_list, key = lambda x: x[0], reverse = True)[:7]
results   


[[0.7697424852230645, 'OverallQual', 'TotRmsAbvGrd'],
 [0.7637046039286673, 'OverallQual', 'GarageArea'],
 [0.7577786618650431, 'OverallQual', '2ndFlrSF'],
 [0.7561427255607789, '2ndFlrSF', 'TotRmsAbvGrd'],
 [0.7560013712047065, '2ndFlrSF', 'GrLivArea'],
 [0.7542316382873494, 'OverallQual', 'Fireplaces'],
 [0.754090052704919, 'OverallCond', 'TotalBsmtSF']]

Write code to include the 7 most important interactions in your data set by adding 7 columns. Name the columns "var1_var2", where var1 and var2 are the two variables in the interaction.

In [10]:
# Your code here
data_int = X.copy()
for interaction in results:
    column_name = interaction[1] + " * " + interaction[2]
    data_int[column_name] = data[interaction[1]]*data[interaction[2]]
data_int.head()
    

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,OverallQual * TotRmsAbvGrd,OverallQual * GarageArea,OverallQual * 2ndFlrSF,2ndFlrSF * TotRmsAbvGrd,2ndFlrSF * GrLivArea,OverallQual * Fireplaces,OverallCond * TotalBsmtSF
0,-0.207142,0.651479,-0.517200,-0.459303,-0.793434,1.161852,0.370333,0.912210,0.351000,-0.951226,0.594286,0.228669,0.756922,1.059852,0.430272,-0.619704,0.237551
1,-0.091886,-0.071836,2.179628,0.466465,0.257140,-0.795163,-0.482512,-0.318683,-0.060731,0.600495,0.022893,0.004363,0.057121,0.253405,0.383676,-0.043137,1.016720
2,0.073480,0.651479,-0.517200,-0.313369,-0.627826,1.189351,0.515013,-0.318683,0.631726,0.600495,-0.207616,0.411557,0.774837,-0.379026,0.612531,0.391210,0.162074
3,-0.096897,0.651479,-0.517200,-0.687324,-0.521734,0.937276,0.383659,0.296763,0.790804,0.600495,0.193335,0.515193,0.610616,0.278149,0.359595,0.391210,0.355484
4,0.375148,1.374795,-0.517200,0.199680,-0.045611,1.617877,1.299326,1.527656,1.698485,0.600495,2.100214,2.335068,2.224249,2.471560,2.102150,0.825557,-0.103274


## Include polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of 4, the particular column is raised to the power of 2 and 3 as well in other terms. We only want to include "pure" polynomials, so make sure no interactions are included. We want the result to return a list that contain tuples of the form:

`(var_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [24]:
# Your code here
poly_list = []
for column in data.columns:
    for n in range(1,5):
        data = X.copy()
        poly = PolynomialFeatures(n, include_bias = False)
        print("1")
        poly_raw = poly.fit_transform([data[column]], n)
        print("2")
        X_poly = pd.DataFrame(poly_raw, )
        print(X_poly.shape , n)
        print("3")
        X_fin = pd.concat([data.drop(column, axis = 1) ,X_poly], axis = 1)
        print(X_fin.shape)
        score = np.mean(cross_val_score(linreg, X_fin , y, scoring = "r2", cv = kfold))
        print("4")
        poly_list.append([column, n, score])
        print(column, n)
        
        
        

1
2
(1, 1460) 1
3
(1460, 1469)
4
LotArea 1
1
2
(1, 1067990) 2
3


KeyboardInterrupt: 

In [12]:
poly_list = []
for column in data.columns:
    for n in range(1,5):
        data = X.copy()
        poly = PolynomialFeatures(n, include_bias = False)
        poly_raw = poly.fit_transform(pd.DataFrame(data[column]), n)
        X_poly = pd.DataFrame(poly_raw)
        X_fin = pd.concat([data.drop(column, axis = 1) ,X_poly], axis = 1)
        score = np.mean(cross_val_score(linreg, X_fin , y, scoring = "r2", cv = kfold))
        poly_list.append([column, n, score])
poly_list

[['LotArea', 1, 0.7524751004088885],
 ['LotArea', 2, 0.7496267647224734],
 ['LotArea', 3, 0.6201223520713423],
 ['LotArea', 4, -0.7581530651707099],
 ['OverallQual', 1, 0.7524751004088885],
 ['OverallQual', 2, 0.7806086732834392],
 ['OverallQual', 3, 0.7790396103063504],
 ['OverallQual', 4, 0.7791919536787744],
 ['OverallCond', 1, 0.7524751004088887],
 ['OverallCond', 2, 0.7517975998498203],
 ['OverallCond', 3, 0.7510436777360656],
 ['OverallCond', 4, 0.752615729949287],
 ['TotalBsmtSF', 1, 0.7524751004088885],
 ['TotalBsmtSF', 2, 0.6756596880082331],
 ['TotalBsmtSF', 3, 0.5060486211450481],
 ['TotalBsmtSF', 4, -12.99031757055092],
 ['1stFlrSF', 1, 0.7524751004088887],
 ['1stFlrSF', 2, 0.7347136602299728],
 ['1stFlrSF', 3, 0.7439482285230936],
 ['1stFlrSF', 4, -4.579869787148838],
 ['2ndFlrSF', 1, 0.7524751004088887],
 ['2ndFlrSF', 2, 0.7711185948668178],
 ['2ndFlrSF', 3, 0.7753364307446551],
 ['2ndFlrSF', 4, 0.7712850562337284],
 ['GrLivArea', 1, 0.7524751004088887],
 ['GrLivArea', 2,

In [13]:
top_df = pd.DataFrame(poly_list)

For each variable, print out the maximum $R^2$ possible when including Polynomials.

In [14]:
# Your code here
print(sorted(top_df.groupby(by = [0]).max()[2], reverse=True))
print(top_df.groupby(by = [0]).max()[2])
print(top_df.groupby(by = [0]).max()[1])





[0.8069252555392874, 0.7806086732834392, 0.7753364307446551, 0.7673361786205585, 0.752615729949287, 0.7525382764131427, 0.7524751004088887, 0.7524751004088887, 0.7524751004088885, 0.7524751004088885]
0
1stFlrSF        0.752475
2ndFlrSF        0.775336
Fireplaces      0.752475
GarageArea      0.767336
GrLivArea       0.806925
LotArea         0.752475
OverallCond     0.752616
OverallQual     0.780609
TotRmsAbvGrd    0.752538
TotalBsmtSF     0.752475
Name: 2, dtype: float64
0
1stFlrSF        4
2ndFlrSF        4
Fireplaces      4
GarageArea      4
GrLivArea       4
LotArea         4
OverallCond     4
OverallQual     4
TotRmsAbvGrd    4
TotalBsmtSF     4
Name: 1, dtype: int64


In [17]:
top2 = ['GrLivArea', "OverallQual"]

Which two variables seem to benefit most from adding polynomial terms?

Add Polynomials for the two features that seem to benefit the most, as in have the best $R^2$ compared to the baseline model. For each of the two features, raise to the Polynomial that generates the best result. Make sure to start from the data set `df_inter` so the final data set has both interactions and polynomials in the model.

In [22]:
# Your code here
data = data_int.copy()
n=4
for column in top2:
    
    poly = PolynomialFeatures(n, include_bias = False)
    poly_raw = poly.fit_transform(pd.DataFrame(data[column]), n)
    columns_new = [column+"_"+str(n) for n in list(range(1,5))]
    X_poly = pd.DataFrame(poly_raw, columns = columns_new)
    X_fin = pd.concat([data.drop(column, axis = 1) ,X_poly], axis = 1)
    

# #     score = np.mean(cross_val_score(linreg, X_fin , y, scoring = "r2", cv = kfold))
# #     poly_list.append([column, n, score])
    
    
    

In [21]:
list(range(1,5))

[1, 2, 3, 4]

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [23]:
# Your code here
X_fin.head()

,LotArea,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,OverallQual * TotRmsAbvGrd,OverallQual * GarageArea,OverallQual * 2ndFlrSF,2ndFlrSF * TotRmsAbvGrd,2ndFlrSF * GrLivArea,OverallQual * Fireplaces,OverallCond * TotalBsmtSF,OverallQual_1,OverallQual_2,OverallQual_3,OverallQual_4
0,-0.207142,-0.517200,-0.459303,-0.793434,1.161852,0.370333,0.912210,0.351000,-0.951226,0.594286,0.228669,0.756922,1.059852,0.430272,-0.619704,0.237551,0.651479,0.424425,0.276504,0.180137
1,-0.091886,2.179628,0.466465,0.257140,-0.795163,-0.482512,-0.318683,-0.060731,0.600495,0.022893,0.004363,0.057121,0.253405,0.383676,-0.043137,1.016720,-0.071836,0.005160,-0.000371,0.000027
2,0.073480,-0.517200,-0.313369,-0.627826,1.189351,0.515013,-0.318683,0.631726,0.600495,-0.207616,0.411557,0.774837,-0.379026,0.612531,0.391210,0.162074,0.651479,0.424425,0.276504,0.180137
3,-0.096897,-0.517200,-0.687324,-0.521734,0.937276,0.383659,0.296763,0.790804,0.600495,0.193335,0.515193,0.610616,0.278149,0.359595,0.391210,0.355484,0.651479,0.424425,0.276504,0.180137
4,0.375148,-0.517200,0.199680,-0.045611,1.617877,1.299326,1.527656,1.698485,0.600495,2.100214,2.335068,2.224249,2.471560,2.102150,0.825557,-0.103274,1.374795,1.890060,2.598445,3.572328


## Full model R-squared

Check out the $R^2$ of the full model.

In [ ]:
# Your code here

## Find the best Lasso regularization parameter

You learned that when using Lasso regularization, your coefficients shrink to 0 when using a higher regularization parameter. Now the question is which value we should choose for the regularization parameter. 

This is where the AIC and BIC come in handy! We'll use both criteria in what follows and perform cross-validation to select an optimal value of the regularization parameter $alpha$ of the Lasso estimator.

Read the page here: https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html and create a similar plot as the first one listed on the page. 

In [ ]:
from sklearn.linear_model import Lasso, LassoCV, LassoLarsCV, LassoLarsIC

In [ ]:
# Your code here 


## Analyze the final result

Finally, use the best value for the regularization parameter according to AIC and BIC, and compare $R^2$ and RMSE using train-test split. Compare with the baseline model.

Remember, you can find the Root Mean Squared Error (RMSE) by setting `squared=False` inside the function (see [the documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)), and the RMSE returns values that are in the same units as our target - so we can see how far off our predicted sale prices are in dollars.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split

In [ ]:
# Split X_scaled and y into training and test sets
# Set random_state to 1
X_train, X_test, y_train, y_test = None

# Code for baseline model
linreg_all = None


# Print R-Squared and RMSE


In [ ]:
# Split df_inter and y into training and test sets
# Set random_state to 1
X_train, X_test, y_train, y_test = None

# Code for lasso with alpha from AIC
lasso = None


# Print R-Squared and RMSE


In [ ]:
# Code for lasso with alpha from BIC
lasso = None


# Print R-Squared and RMSE


## Level up (Optional)

### Create a Lasso path

From this section, you know that when using Lasso, more parameters shrink to zero as your regularization parameter goes up. In Scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for subset selection
This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to create better linear models and how to use AIC and BIC for both feature selection and to optimize your regularization parameter when performing Ridge and Lasso. 